In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
import pandas as pd
import numpy as np
import json

In [2]:
!ls

app.py
data_engineering.ipynb
data_engineering_updated.ipynb
flask_queries.ipynb
info_data
raw_data
README.md
refugees_and_conflict.sqlite
static
templates


In [3]:
engine = create_engine('sqlite:///refugees_and_conflict.sqlite')

In [4]:
Base = automap_base()

In [5]:
Base.prepare(engine, reflect = True)

In [6]:
Base.classes.keys()

['asylum', 'battle_deaths', 'info_table', 'origin']

In [7]:
session = Session(engine)

In [8]:
Asylum = Base.classes.asylum
BattleDeaths = Base.classes.battle_deaths
Origin = Base.classes.origin
InfoTable = Base.classes.info_table

In [9]:
samples_df = pd.read_sql_table('asylum', engine)
samples_df.head()

,id,country_name,country_lat,country_lon,country_code,y1990,y1991,y1992,y1993,y1994,...,y2008,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017
0,1,Albania,41.0000,20.0,ALB,0,0,3000,3000,3000,...,65,70,76,82,86,93,104,104,111,0
1,2,Algeria,28.0000,3.0,DZA,169110,169124,219314,219067,219073,...,94093,94137,94144,94148,94133,94150,94128,94182,94220,0
2,3,American Samoa,-14.3333,-170.0,ASM,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Andorra,42.5000,1.6,AND,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Angola,-12.5000,18.5,AGO,11557,11022,11002,10878,10686,...,12710,14734,15155,16223,23413,23783,15474,15555,15537,0


## Query Country Names

In [10]:
country_names = session.query(Asylum.country_name).all()
country_names = [x[0] for x in country_names]
country_names

['Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Polynesia',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Greenland',
 'Grenada',
 'Guam',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hun

## Query Info Table

In [11]:
samples_df = pd.read_sql_table('info_table', engine)
samples_df.head()

,id,country_name,gdp_YR2015,population_YR2016,asylum_YR2016,origin_YR2016
0,1,Afghanistan,19215562178.9798,34656,17980,2501410
1,2,Albania,11335264966.561,2876,0,11050
2,3,Algeria,165874330876.321,40606,86,3675
3,4,American Samoa,659000000,56,0,0
4,5,Andorra,2811489408.89431,77,0,0


In [12]:
first_row_samples = session.query(InfoTable).first()
first_row_samples.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1f75480ce10>,
 'asylum_YR2016': 17980,
 'country_name': 'Afghanistan',
 'gdp_YR2015': '19215562178.9798',
 'id': 1,
 'origin_YR2016': 2501410,
 'population_YR2016': '34656'}

In [19]:
info_table = session.query(InfoTable.country_name, InfoTable.gdp_YR2015, InfoTable.population_YR2016, 
                           InfoTable.asylum_YR2016, InfoTable.origin_YR2016).all()

In [20]:
info_table = pd.DataFrame(info_table)
info_table.head()

,country_name,gdp_YR2015,population_YR2016,asylum_YR2016,origin_YR2016
0,Afghanistan,19215562178.9798,34656,17980,2501410
1,Albania,11335264966.561,2876,0,11050
2,Algeria,165874330876.321,40606,86,3675
3,American Samoa,659000000,56,0,0
4,Andorra,2811489408.89431,77,0,0


In [21]:
info_table.columns = ['country_name', 'GDP YEAR 2015', 'POPULATION YEAR 2016', 'ASYLUM NUMBER YEAR 2016', 'REFUGEE ORIGIN YEAR 2016']

In [22]:
info_table = info_table.set_index('country_name').to_dict('index')

In [23]:
info_table

{'Afghanistan': {'ASYLUM NUMBER YEAR 2016': 17980,
  'GDP YEAR 2015': '19215562178.9798',
  'POPULATION YEAR 2016': '34656',
  'REFUGEE ORIGIN YEAR 2016': 2501410},
 'Albania': {'ASYLUM NUMBER YEAR 2016': 0,
  'GDP YEAR 2015': '11335264966.561',
  'POPULATION YEAR 2016': '2876',
  'REFUGEE ORIGIN YEAR 2016': 11050},
 'Algeria': {'ASYLUM NUMBER YEAR 2016': 86,
  'GDP YEAR 2015': '165874330876.321',
  'POPULATION YEAR 2016': '40606',
  'REFUGEE ORIGIN YEAR 2016': 3675},
 'American Samoa': {'ASYLUM NUMBER YEAR 2016': 0,
  'GDP YEAR 2015': '659000000',
  'POPULATION YEAR 2016': '56',
  'REFUGEE ORIGIN YEAR 2016': 0},
 'Andorra': {'ASYLUM NUMBER YEAR 2016': 0,
  'GDP YEAR 2015': '2811489408.89431',
  'POPULATION YEAR 2016': '77',
  'REFUGEE ORIGIN YEAR 2016': 0},
 'Angola': {'ASYLUM NUMBER YEAR 2016': 0,
  'GDP YEAR 2015': '102962245246.708',
  'POPULATION YEAR 2016': '28813',
  'REFUGEE ORIGIN YEAR 2016': 8363},
 'Antigua and Barbuda': {'ASYLUM NUMBER YEAR 2016': 0,
  'GDP YEAR 2015': '136